In [1]:
import Levenshtein

def error_rate(a, b):
    msd = Levenshtein.distance(a,b)
    
    return 100.0*msd/max(len(a), len(b))


In [ ]:
import numpy as np
import ast, dataIO, common_functions
from sklearn import cross_validation, linear_model, preprocessing, svm
from sklearn.grid_search import GridSearchCV
reload(dataIO)
reload(common_functions)

def learn_offset(points, targets):
    regr = linear_model.LinearRegression()
    regr.fit(points, targets)
    
    return regr
                
userId = 8
keys = postures.keys()
locations = []
bod = []
targets_x = []
targets_y = []
y = []
touch_centers = []

posture = 0
for key in keys:
    filenos = postures[key]
    if key == "two_hand":
        a, b, c, d, e, f = dataIO.process_twohand(userId, posture)
        posture += 2
    else:
        a, b, c, d, e, f = dataIO.process_posture(userId, filenos, posture)
        posture += 1
    
    locations += a
    bod += b
    targets_x += c 
    targets_y += d 
    y += e
    touch_centers += f

locations = np.array(locations)
bod = np.array(bod)
targets_x = np.array(targets_x)
targets_y = np.array(targets_y)
y = np.array(y)
touch_centers = np.array(touch_centers)

scaler = preprocessing.StandardScaler().fit(bod)  
bod_scaled = scaler.transform(bod)

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1, 0.1 ,1e-2, 1e-3, 1e-4],
                     'C': [0.001, 0.01, 0.1 ,1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [0.001, 0.01, 0.1 ,1, 10, 100, 1000]}]

clf = GridSearchCV(svm.SVC(C=1, cache_size=500), tuned_parameters)
#clf = svm.SVC(C=100, kernel='rbf', gamma=0.01, cache_size=500)
clf.fit(bod_scaled, y)

regr_x = []
regr_y = []

for i in range(0,5):
    index = np.where(y==i)[0]
    regr_x.append(learn_offset(locations[index], targets_x[index]))
    regr_y.append(learn_offset(locations[index], targets_y[index]))


    
    
new_points = []
for i in range(len(points_test)):

    point = points_test[i]
    bod_data = scaler.transform(bod_test[i])
    pred = clf.predict(bod_data)

    pred_x = regr_x[pred].predict(point)
    pred_y = regr_y[pred].predict(point)

    new_points.append([point[0]+pred_x, point[1]+pred_y])


new_points = np.array(new_points).T
centers_test = centers_test.T

    